In [ ]:
!pip install transformers==4.2.2

     |████████████████████████████████| 1.8 MB 8.7 MB/s 
     |████████████████████████████████| 2.9 MB 60.5 MB/s 
     |████████████████████████████████| 895 kB 66.9 MB/s 


In [ ]:
!nvidia-smi

Tue Sep 14 08:43:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |  15090MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prepare the dataset and build a ``TextDataset``

The next step is to extract the instructions from all recipes and build a `TextDataset`. The `TextDataset` is a custom implementation of the [Pytroch `Dataset` class](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class) implemented by the transformers library. If you want to know more about Dataset in Pytroch you can check out this [youtube video](https://www.youtube.com/watch?v=PXOzkkB5eH0&ab_channel=PythonEngineer).

First, we are going to split the `recipes.json` into a `train` and `test` section and extract `Instructions` from the recipes and write them into a `train_dataset.txt` and `test_dataset.txt`

In [ ]:
with open('/content/стищки.txt','r',encoding='windows-1251') as f:
    lines = f.readlines()

In [ ]:
for line in lines:
  print(line)

In [ ]:
fl = open('test.txt', 'w')
fl.writelines(lines)
fl.close()

In [ ]:
with open('test.txt','r') as f:
    lines = f.readlines()

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


with open('/content/test.txt') as f:
    lines = f.readlines()

def build_text_files(lines, dest_path):
    """
    name:name \n
    poem:start
    poem \n
    poem:end

    """
    data = []
    for i in range(len(lines)):
      line = lines[i]
      if not line.startswith('\t\t') and len(line)>4 and line.find('* * *')!=0:
          data+=['poem:end' + '\n' +'name:'+(line)+'\n' + 'poem:start']
          
      else:
        
        data+=[line.replace('–\xa0','').replace('»','').replace('«','')]
      
    
    
    fl = open(dest_path, 'w')
    fl.writelines(data)
    fl.close()

train, test = train_test_split(lines,test_size=0.1) 


build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 23911
Test dataset length: 2657


the next step is to download the tokenizer, which we use. We use the tokenizer from the `german-gpt2` model on [huggingface](https://huggingface.co/anonymous-german-nlp/german-gpt2).

In [ ]:
from transformers import AutoTokenizer
BACKBONE = 'sberbank-ai/rugpt3small_based_on_gpt2'

tokenizer = AutoTokenizer.from_pretrained(BACKBONE)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
special_tokens_dict = {
        "additional_special_tokens": [
            'name:', 'poem:start', 'poem:end'
        ]
    }
tokenizer.add_special_tokens(special_tokens_dict)

0

In [ ]:
tokenizer.special_tokens_map

{'additional_special_tokens': "['name:', 'poem:start', 'poem:end']",
 'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [12]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=50)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=50)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset('/content/test.txt','/content/test.txt',tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [13]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
BACKBONE = 'sberbank-ai/rugpt3small_based_on_gpt2'
model = AutoModelWithLMHead.from_pretrained(BACKBONE)


training_args = TrainingArguments(
    output_dir="./pushn", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 100, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    fp16_opt_level = 'O4',
    )


trainer = Trainer(
    model=model,
    args=training_args,
    
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


# Train and save the model

To train the model we can simply run `Trainer.train()`.

In [14]:
trainer.train()

RuntimeError: ignored

In [15]:
!nvidia-smi

Tue Sep 14 08:51:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |  15098MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model()

In [ ]:
tokenizer.save_pretrained('/content/pushn')

('/content/pushn/tokenizer_config.json',
 '/content/pushn/special_tokens_map.json',
 '/content/pushn/vocab.json',
 '/content/pushn/merges.txt',
 '/content/pushn/added_tokens.json')

#Simple api example

In [ ]:
import numpy as np
import torch
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from starlette.middleware import Middleware
from starlette.middleware.cors import CORSMiddleware

np.random.seed(42)
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2Tokenizer
BACKBONE = "sberbank-ai/rugpt3small_based_on_gpt2"
path_model = "content/pushn"
device='cuda'

app = FastAPI()



special_tokens_dict = {
        "additional_special_tokens": [
            'name:', 'poem:start', 'poem:end'
        ]
    }


tok = GPT2Tokenizer.from_pretrained(BACKBONE)
tok.add_special_tokens(special_tokens_dict)

model = GPT2LMHeadModel.from_pretrained(path_model).to(device)

def push(name,lenght=200,temp=1.1,num=2):
  
  repetition_penalty = 2.6
  temperature = temp
  top_k =4 

  inpt = tok.encode('name:'+name+'poem:start', return_tensors="pt").to(device)

  max_length=lenght

  out = model.generate(inpt,max_length= max_length,
                       do_sample=True, top_k=5000, top_p=0.95, 
                       temperature=temperature,
                       stop_token='poem:end')
  decoded = tok.decode(out[0]).replace('>','').replace('<','')
  return decoded


origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
@app.get("/")
def main():
    return "go to /stih/?name=пидр"



@app.get('/push/')
def detect_spam_query(name: str, temp: float, length:int):
  
  return {"text":push(text,temp=temp)}

import nest_asyncio
#from pyngrok import ngrok
import uvicorn
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=5001,timeout_keep_alive=10000)


In [ ]:
print(push(name='Памятник'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


name: Памятник poem:start  у Бородина,[70 - На полотне, снятом по берлинскому приказу, изображен местный царь, которому выпадает в удел Жермий Тар.Гражданская война, глава I.]

		Венок мраморных новых богачей
		Из тусклой кучки блестящих ассигнаций.
		И царевич в ленте боярской
		Элладиеву ударя семя,
		Со скипетром лавровым
		Не дает созреть царствам терпенью.
		Свой царизм на деревьях берет,
		Мечте равнодушно рад —
		Недруги царя своевольного,
		Всяк свободный миг превраща,
		Тебя с улыбкой готовы принять,
		До самой вершины тебя не отважишь.
		Как мило стяжать венцы – скипетр, скипетр императора!
		Ц


In [ ]:
!zip -r stihi.zip /content/pushn

  adding: content/pushn/ (stored 0%)
  adding: content/pushn/vocab.json (deflated 74%)
  adding: content/pushn/config.json (deflated 48%)
  adding: content/pushn/added_tokens.json (deflated 26%)
  adding: content/pushn/training_args.bin (deflated 46%)
  adding: content/pushn/tokenizer_config.json (deflated 66%)
  adding: content/pushn/pytorch_model.bin (deflated 16%)
  adding: content/pushn/special_tokens_map.json (deflated 68%)
  adding: content/pushn/merges.txt (deflated 76%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
